#### Search Engine with Tools and Agents

In [1]:
#### Arvix-- Research
### Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [83]:
api_wrapper_wiki= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki= WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [85]:
api_wrapper_arxiv= ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv= ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [86]:
tools=[wiki,arxiv]

In [87]:
## custom tools[RAG TOOL]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/2024_United_States_presidential_election")
docs=loader.load()
documents= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb= FAISS.from_documents(documents,OpenAIEmbeddings())
retriever= vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBCE822200>, search_kwargs={})

In [89]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool= create_retriever_tool(retriever,"Election NEWS","Search any information about elections")
retriever_tool.name

'Election NEWS'

In [90]:
tools =[wiki, arxiv,retriever_tool]

In [91]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'o:\\Langchain_project\\1-Q&A Chatbot\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Election NEWS', description='Search any information about elections', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DBD8416950>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBCE822200>

In [92]:
## run all this tools with agents and LLM Models
### Tools, LLM--> AgentExecutor

from dotenv import load_dotenv
from langchain_groq import ChatGroq
import openai
load_dotenv()
import os
groq_api_key= os.getenv("GROQ_API_KEY")
openai.api_key= os.getenv("OPENAI_API_KEY")

llm= ChatGroq(groq_api_key=groq_api_key, model_name= "Llama3-8b-8192")

In [93]:
### Prompt Template
from langchain import hub
prompt= hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [94]:
### Agents 
from langchain.tools import Tool
from langchain.agents import create_openai_tools_agent
agent= create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [95]:
## AGENT Excuter
from langchain.agents import AgentExecutor
agent_excutor= AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_excutor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [96]:
agent_excutor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was foundIt seems like the tool "wikipedia" didn't return a relevant result. Let me try again.

Can you please provide more context or information about Langsmith? What do you mean by Langsmith? Is it a programming language, a software, or something else? Providing more details will help me better understand your question and provide a more accurate answer.

> Finished chain.


{'input': 'Tell me about Langsmith',
 'output': 'It seems like the tool "wikipedia" didn\'t return a relevant result. Let me try again.\n\nCan you please provide more context or information about Langsmith? What do you mean by Langsmith? Is it a programming language, a software, or something else? Providing more details will help me better understand your question and provide a more accurate answer.'}